# Attempt to combine input of different shapes in sklearn
The model will have two inputs: one for time-dependent data (client purchases) and one for time-independent data (client data).

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklift.models import ClassTransformation
from sklift import metrics
from sklift.viz import plot_qini_curve
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier

In [3]:
import chime

In [4]:
%load_ext chime

In [19]:
class InputSelect():
    def __init__(self, n_input):
        self.n_input = n_input
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return X[self.n_input]
        


In [31]:
clf0 =  Pipeline([('select', InputSelect(0)), ('xgb', XGBClassifier())])

In [32]:
clf1 =  Pipeline([('select', InputSelect(1)), ('mlp', MLPClassifier(hidden_layer_sizes=(4,)))])

In [14]:
X0 = np.random.rand(5, 3)
X0

array([[0.10192232, 0.71399311, 0.94919603],
       [0.84238366, 0.05693435, 0.65088989],
       [0.44000488, 0.36154033, 0.85774531],
       [0.70641324, 0.2638742 , 0.29337737],
       [0.78292424, 0.40313449, 0.14893677]])

In [15]:
X1 = np.random.rand(5, 2, 4)

In [11]:
X = (X0, X1)

In [12]:
treat = np.array([0, 1, 0, 1, 0])

In [13]:
y = np.array([1, 1, 1, 0, 0])

In [28]:
clf = VotingClassifier(estimators=[('clf0', clf0), ('clf1', clf1)],
                      voting='soft')

In [33]:
clf1.fit(X, y)

ValueError: Found array with dim 3. MLPClassifier expected <= 2.

# Found that MLP in sklearn only works with 2-dim data.